<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-03 13:44:48
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 52.31 K (0.38%)
Current PnL: -13.07 L (-9.11%)
CY Booked + Current PnL: -6.33 L (-4.41%)
-------------------
Total profit:  4.83 L
Total loss:  -17.90 L
-------------------
Total Booked + Current PnL: 20.65 L (17.72%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.93%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.49 L (52.25%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,93.75,133677.0,-8802.0,9424.0,-0.78,-6.18,7.05,0.44,243.0,-0.93,0.98,57.78,OX40N,NTT,BANKS
39,HINDALCO,651.95,761.55,-19.08,H-LC,86.46,111736.0,7424.0,10112.0,0.58,7.12,9.05,16.81,27.0,0.73,0.82,24.31,X5K,ATH,METALS
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,37.50,102098.0,23736.0,10291.0,-0.41,30.29,10.08,43.43,77.0,2.31,0.75,37.22,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-16.52,H-LC,88.54,113745.0,10179.0,10533.0,1.91,9.83,9.26,20.00,30.0,0.97,0.83,25.29,X40N,BTT,HEALTHCARE
71,SBIN,760.30,863.00,-10.56,M-LC,34.38,215485.0,14006.0,13209.0,0.03,6.95,6.13,13.51,76.0,1.06,1.58,20.42,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142372.0,530.0,56038.0,-0.00,0.37,39.36,39.88,259.0,0.01,1.04,44.28,XY24,NTT,POWER
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,46.88,151725.0,812.0,59643.0,2.15,0.54,39.31,40.06,102.0,0.01,1.11,24.10,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,HONAUT,42646.15,58357.33,-15.24,X-MC,61.46,122160.0,-5778.0,52907.0,4.91,-4.52,43.31,36.84,56.0,-0.11,0.89,25.06,X40N,ATH,ELECTRICAL
15,BLUESTARCO,1646.70,2337.55,-2.36,H-SC,97.92,183110.0,18440.0,50648.0,4.27,11.20,27.66,41.95,121.0,0.36,1.34,19.50,X40N,ATH,AC
93,VOLTAS,1278.28,1918.49,-24.24,H-MC,89.58,206190.0,14448.0,81589.0,3.14,7.54,39.57,50.08,100.0,0.18,1.51,14.84,XY25,ATH,AC
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,46.88,151725.0,812.0,59643.0,2.15,0.54,39.31,40.06,102.0,0.01,1.11,24.10,AR,ATH,AUTO
36,HAVELLS,1588.02,2069.16,-4.46,X-MC,62.50,246948.0,-783.0,75838.0,2.10,-0.32,30.71,30.30,53.0,-0.01,1.81,12.93,X40,ATH,ELECTRICAL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,1.04,16326.0,-33267.0,2189.0,-2.78,-67.08,13.41,-62.66,NaN,-15.20,0.12,9.49,XY24,ATH,MISC
78,SURYODAY,189.78,240.00,51.32,H-SC,10.42,107914.0,-42012.0,81691.0,-2.57,-28.02,75.70,26.46,145.0,-0.51,0.79,38.25,XR,NTT,BANKS
44,ICICIPRULI,600.83,790.00,-9.70,H-MC,36.46,145759.0,9971.0,32781.0,-2.10,7.34,22.49,31.48,119.0,0.30,1.07,20.35,X40,ATH,INSURANCE
11,BANDHANBNK,214.25,400.00,-28.38,H-SC,79.17,213684.0,-34846.0,250309.0,-1.95,-14.02,117.14,86.70,129.0,-0.14,1.56,41.36,XY24,NTT,BANKS
28,DIGITIDE,188.38,237.72,NaN,NaN,6.25,46954.0,5699.0,5109.0,-1.72,13.81,10.88,26.19,1.0,1.12,0.34,10.54,XY24,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,93.75,133677.0,-8802.0,9424.0,-0.78,-6.18,7.05,0.44,243.0,-0.93,0.98,57.78,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-30.89,M-SC,8.33,147283.0,-23858.0,23801.0,1.59,-13.94,16.16,-0.03,189.0,-1.00,1.08,5.86,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,91.67,148800.0,-25759.0,25787.0,0.53,-14.76,17.33,0.02,127.0,-1.00,1.09,29.03,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-16.81,X-MC,19.79,87360.0,-15292.0,15288.0,0.89,-14.90,17.50,-0.00,58.0,-1.00,0.64,32.80,OX40N,NTT,IT
92,VIPIND,488.80,489.0,-212.13,H-SC,94.79,80925.0,-14391.0,14429.0,-1.19,-15.10,17.83,0.04,154.0,-1.00,0.59,59.62,OX40N,NTT,MISC


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,6.25,46954.0,5699.0,5109.0,-1.72,13.81,10.88,26.19,1.0,1.12,0.34,10.54,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,41.67,266651.0,-29283.0,123193.0,-0.14,-9.90,46.20,31.73,1.0,-0.24,1.95,6.73,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,64.58,291663.0,30062.0,115557.0,1.17,11.49,39.62,55.67,2.0,0.26,2.13,18.30,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-16.87,X-LC,29.17,260254.0,-10093.0,44712.0,0.72,-3.73,17.18,12.81,6.0,-0.23,1.90,8.84,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.46,X-LC,33.33,280894.0,15468.0,41432.0,0.54,5.83,14.75,21.44,9.0,0.37,2.05,12.39,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,27.08,171914.0,31074.0,13409.0,-1.43,22.06,7.80,31.59,89.0,2.32,1.26,49.33,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,1.67,M-LC,37.50,102098.0,23736.0,10291.0,-0.41,30.29,10.08,43.43,77.0,2.31,0.75,37.22,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,72.92,73425.0,38976.0,969.0,-0.48,113.14,1.32,115.97,NaN,40.22,0.54,12.28,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,96.87,M-MC,63.54,236362.0,11400.0,156259.0,-0.16,5.07,66.11,74.53,185.0,0.07,1.73,37.23,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,26.30,H-SC,68.75,129175.0,3878.0,55326.0,-0.43,3.10,42.83,47.25,139.0,0.07,0.94,61.61,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.09,H-SC,44.79,119283.0,17279.0,27745.0,0.69,16.94,23.26,44.14,124.0,0.62,0.87,35.93,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,37.50,102098.0,23736.0,10291.0,-0.41,30.29,10.08,43.43,77.0,2.31,0.75,37.22,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,27.08,171914.0,31074.0,13409.0,-1.43,22.06,7.80,31.59,89.0,2.32,1.26,49.33,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,72.92,73425.0,38976.0,969.0,-0.48,113.14,1.32,115.97,NaN,40.22,0.54,12.28,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,90.62,205609.0,33988.0,128074.0,0.84,19.80,62.29,94.43,198.0,0.27,1.50,51.75,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,83.33,167462.0,16517.0,92941.0,1.16,10.94,55.50,72.51,69.0,0.18,1.22,15.35,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.89,H-SC,82.29,135871.0,12535.0,122297.0,-0.23,10.16,90.01,109.32,122.0,0.10,0.99,34.23,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,26.30,H-SC,68.75,129175.0,3878.0,55326.0,-0.43,3.10,42.83,47.25,139.0,0.07,0.94,61.61,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68045.0,-45504.0,85553.0,-0.22,-40.07,125.73,35.27,261.0,-0.53,0.50,66.54,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-19.15,H-SC,57.29,91935.0,-6045.0,28785.0,-0.39,-6.17,31.31,23.21,137.0,-0.21,0.67,33.34,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,8.44,X-SC,98.96,100930.0,-33.0,94561.0,0.03,-0.03,93.69,93.63,64.0,-0.00,0.74,50.33,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,1.67,M-LC,37.50,102098.0,23736.0,10291.0,-0.41,30.29,10.08,43.43,77.0,2.31,0.75,37.22,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-355.15,H-SC,4.17,108550.0,-24736.0,109332.0,0.28,-18.56,100.72,63.47,148.0,-0.23,0.79,81.96,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-29.18,L-SC,100.00,142372.0,530.0,56038.0,-0.00,0.37,39.36,39.88,259.0,0.01,1.04,44.28,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,8.44,X-SC,98.96,100930.0,-33.0,94561.0,0.03,-0.03,93.69,93.63,64.0,-0.00,0.74,50.33,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-2.36,H-SC,97.92,183110.0,18440.0,50648.0,4.27,11.20,27.66,41.95,121.0,0.36,1.34,19.50,X40N,ATH,AC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68045.0,-45504.0,85553.0,-0.22,-40.07,125.73,35.27,261.0,-0.53,0.50,66.54,XR,NTT,HOTELS
51,JIOFIN,310.58,387.00,6.78,H-LC,95.83,228865.0,11459.0,42043.0,0.08,5.27,18.37,24.61,95.0,0.27,1.67,62.73,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.95
1,25,41.99
2,50,70.51


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.39
LC    34.00
MC    21.61
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.53
X40      12.26
XY25     12.03
X40N     11.88
XR       10.07
AR        8.22
OX40N     8.21
MH        1.69
X5K       1.48
X200      1.40
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.57
M-SC    15.97
H-LC    14.13
X-LC    12.16
H-MC     9.55
X-MC     7.59
M-LC     6.66
X-SC     4.41
M-MC     4.11
L-SC     2.44
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.06,-1.47,35.38
IT,12.11,-9.56,63.37
BANKS,7.86,-6.18,49.50
FINANCE,7.85,-12.16,52.75
MISC,6.61,-15.03,68.41
PAINTS,5.74,-11.36,33.31
ELECTRICAL,5.53,2.40,43.18
HEALTHCARE,4.56,-3.40,33.60
AUTO,4.35,-13.56,63.49


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2691219.0,27
XR,1079332.0,13
AR,1020347.0,10
X40,643321.0,9
X40N,537896.0,11
XY25,473147.0,8
OX40N,308729.0,11
SR,176026.0,2
MH,84294.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1970643.0,20
M-SC,1759054.0,21
H-LC,607592.0,12
H-MC,529053.0,8
X-LC,510993.0,7
X-MC,412998.0,6
M-MC,408550.0,3
X-SC,315408.0,5
M-LC,263653.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      867160.0      6
M-SC       XY24      814833.0      7
H-SC       AR        522011.0      3
M-SC       XR        332289.0      4
M-MC       XY24      298309.0      2
H-MC       XY24      296146.0      3
X-LC       X40       238750.0      2
H-SC       XR        219808.0      3
           X40N      216478.0      4
H-LC       X40N      196893.0      4
M-SC       AR        181627.0      2
           OX40N     162914.0      5
M-LC       XY24      157503.0      3
H-LC       X40       153774.0      3
L-SC       XR        147675.0      2
X-MC       X40       146923.0      2
X-SC       XR        123061.0      1
X-LC       AR        116353.0      1
M-SC       XY25      114833.0      1
X-LC       XY25      114198.0      3
M-MC       XR        110241.0      1
X-MC       XY25      103110.0      1
           XY24       94770.0      1
X-SC       SR         94561.0      1
M-LC       XR         92941.0      1
H-SC       OX40N      89860.0      3
H-MC       XY25       81589.0      1
M-SC       SR         81465.0      1
H-LC       AR         79917.0      1
M-SC       X40        71093.0      1
H-LC       X200       68405.0      1
           X5K        66560.0      2
L-SC       AR         59827.0      1
H-MC       AR         59643.0      1
X-SC       XY24       57119.0      1
L-SC       XY24       56038.0      1
H-SC       MH         55326.0      1
L-MC       XR         53317.0      1
X-MC       X40N       52907.0      1
L-LC       XY25       46208.0      1
H-LC       XY24       42043.0      1
X-LC       X40N       41692.0      1
X-SC       OX40N      40667.0      2
H-MC       X40        32781.0      1
           X40N       29926.0      1
           MH         28968.0      1
X-MC       OX40N      15288.0      1
M-LC       XY25       13209.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
